# RL Assignment 2020: NetHack
By Marc Marsden (1437889), Kutlwano Mokoena (1514206), and Tristan Nagan (1484020)

## Colab and Github Instructions
The Notebook is already saved in the repo. To commit a change:<br>
File->Save a copy in Github<br>
The first time you will prompted to give Colab permission to commit changes to your Github. Thereafter you will have to select the repo, write a commit message and then click save. <br>
**NB: Everytime you save to Github you must manually select the correct repo, Colab will not remember the correct repo from the last commit!**

## Setup NetHack

In [1]:
# install prerequisites for nle
!sudo apt-get install -y build-essential autoconf libtool pkg-config \
    python3-dev python3-pip python3-numpy git libncurses5-dev \
    libzmq3-dev flex bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-numpy is already the newest version (1:1.13.3-2ubuntu1).
python3-numpy set to manually installed.
git is already the newest version (1:2.17.1-1ubuntu0.7).
libncurses5-dev is already the newest version (6.1-1ubuntu1.18.04).
python3-dev is already the newest version (3.6.7-1~18.04).
libzmq3-dev is already the newest version (4.2.5-1ubuntu0.2).
The following additional packages will be installed:
  automake autotools-dev file libbison-dev libfl-dev libfl2 libmagic-mgc
  libmagic1 libsigsegv2 m4 python-pip-whl python3-asn1crypto
  python3-cffi-backend python3-crypto python3-cryptography python3-idna
  python3-keyring python3-keyrings.alt python3-pkg-resources
  python3-secretstorage python3-setuptools python3-six python3-wheel
  python3-xdg
Suggested packages:
  autoco

In [2]:
# download, build and install flatbuffers

!git clone https://github.com/google/flatbuffers.git
# all these commands have to be run in the same directory and !cd doesn't change
# the directory permanently in colab see: 
# https://stackoverflow.com/questions/48298146/changing-directory-in-google-colab-breaking-out-of-the-python-interpreter
!cd flatbuffers && cmake -G "Unix Makefiles" && make && sudo make install

Cloning into 'flatbuffers'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 18898 (delta 7), reused 4 (delta 3), pack-reused 18868
Receiving objects: 100% (18898/18898), 11.37 MiB | 7.02 MiB/s, done.
Resolving deltas: 100% (13140/13140), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
-- L

In [3]:
# the next step requires a version of cmake > 3.14.0
!pip install cmake==3.15.3

     |████████████████████████████████| 16.5MB 1.5MB/s 
  Found existing installation: cmake 3.12.0
    Uninstalling cmake-3.12.0:
      Successfully uninstalled cmake-3.12.0


In [4]:
# add -v for verbose if there are any errors
!pip install nle 

     |████████████████████████████████| 6.7MB 2.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/92/a5/796ccb611a92cbdf1f6eaa9b2ec9458d5784a7b909efdf8e8afaa3861783/pybind11-2.6.0-py2.py3-none-any.whl
  Created wheel for nle: filename=nle-0.5.2-cp36-cp36m-linux_x86_64.whl size=2836553 sha256=c285746d903d8f680dd4e164006b5aa558d2932306ac3b02ed0384fffaf862e1
  Stored in directory: /root/.cache/pip/wheels/3c/e1/2e/ece6eaf3e2382f04888155cb96fcb005f94c988d5f0c315d00
Successfully built nle


In [5]:
import nle

In [6]:
import gym
env = gym.make("NetHackScore-v0")
env.reset()
env.step(1)
env.render()

b"It's a wall."
an uncursed +2 pair of leather gloves (being worn)                              
an uncursed +1 robe (being worn)                                                
a blessed spellbook of sleep                                                    
an uncursed scroll of gold detection                                            
3 uncursed potions of healing                                                   
5 uncursed food rations                                                         
6 uncursed apples                                                               
7 uncursed oranges                                                              
3 uncursed fortune cookies                                                      
                                                                               
                                                                               
                                                                               
               